# Wavelength calibration

In [1]:
import numpy as np
import glob
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'


### Loading crispy's dependencies

In [2]:
import sys
codefolder = '../../../../crispy'
if codefolder not in sys.path: sys.path.append(codefolder)
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.params import Params
codefolder = '../../../crispy'
par = Params(codefolder)

### Generate monochromatic wavelengths

In [3]:
# Suppose we create a folder in crispy/ReferenceFiles/ called wavecalR50_770_test
par.wavecalDir = par.prefix+'/wavecalR50_770/'
# construct monochromatic wavelengths, doesn't really matter which wavelengths
lamlist = np.arange(700,845,10)
print(lamlist)


[700 710 720 730 740 750 760 770 780 790 800 810 820 830 840]


In [4]:
# construct images for each wavelength
from crispy.IFS import createWavecalFiles
createWavecalFiles(par,lamlist)

crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming slices are evenly spread in wavelengths
[ 699.  701.]
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming slices are evenly spread in wavelengths
[ 709.  711.]


crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770/det_710.fits
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming slices are evenly spread in wavelengths
[ 719.  721.]
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 3 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770/det_720.fits
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming slices are evenly spread in wavelengths
[ 729.  731.]
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy -

### Loading wavelengths/filelists

In [5]:
#par.lamlist = np.loadtxt(par.wavecalDir + "lamsol.dat")[:, 0]
par.lamlist=lamlist
par.filelist = np.sort(glob.glob(par.wavecalDir + "det*.fits"))

### Run calibration. Choose parameters wisely.

In [6]:
from crispy.tools.wavecal import buildcalibrations
buildcalibrations(par,
                  order=3, # Order of polynomial map of the PSFLets (between 1 and 5)
                  upsample=10, # Upsampling factor of the PSFLets, needs to be at least 5
                  nsubarr=1, # How many zones on the detector have their own PSFLets? 
                  genwavelengthsol=True, # Compute wavelength at the center of all pixels (need to do this at least once)
                  makehiresPSFlets=True, # Compute hires PSFLets (usually False)
                  parallel=True) # Use all computer cores

crispy - INFO - Building calibration files, placing results in ../../../crispy/ReferenceFiles/wavecalR50_770/
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - Initializing PSFlet location transformation coefficients
crispy - INFO - Performing initial optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770/det_710.fits
crispy - INFO - Initializing transformation coefficients with previous values
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ../../../cris

In [7]:
from crispy.tools.reduction import calculateWaveList
lam_mid,lam_end = calculateWaveList(par)
print lam_mid,lam_end


crispy - INFO - Reduced cube will have 19 wavelength bins
[ 703.68421053  711.05263158  718.42105263  725.78947368  733.15789474
  740.52631579  747.89473684  755.26315789  762.63157895  770.
  777.36842105  784.73684211  792.10526316  799.47368421  806.84210526
  814.21052632  821.57894737  828.94736842  836.31578947] [ 700.          707.36842105  714.73684211  722.10526316  729.47368421
  736.84210526  744.21052632  751.57894737  758.94736842  766.31578947
  773.68421053  781.05263158  788.42105263  795.78947368  803.15789474
  810.52631579  817.89473684  825.26315789  832.63157895  840.        ]
13.11653924


In [6]:
from crispy.unitTests import testCreateFlatfield
# par.savePoly = True
# par.saveRotatedInput = True
testCreateFlatfield(par)

crispy - INFO - Reduced cube will have 44 wavelength bins
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming endpoints wavelist is given
[ 700.          703.18181818  706.36363636  709.54545455  712.72727273
  715.90909091  719.09090909  722.27272727  725.45454545  728.63636364
  731.81818182  735.          738.18181818  741.36363636  744.54545455
  747.72727273  750.90909091  754.09090909  757.27272727  760.45454545
  763.63636364  766.81818182  770.          773.18181818  776.36363636
  779.54545455  782.72727273  785.90909091  789.09090909  792.27272727
  795.45454545  798.63636364  801.81818182  805.          808.18181818
  811.36363636  814.54545455  817.72727273  820.90909091  824.09090909
  827.27272727  830.45454545  833.63636364  836.81818182  840.        ]
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 39 secon

In [8]:
from crispy.IFS import reduceIFSMap
tmp=reduceIFSMap(par,par.unitTestsOutputs+'/flatfield.fits',method='optext')


crispy - INFO - Read data from HDU 1 of ../../../crispy/unitTestsOutputs/flatfield.fits
crispy - INFO - Reduced cube will have 19 wavelength bins
crispy - INFO - Writing data to ../../../crispy/SimResults/flatfield_red_optext.fits
crispy - INFO - Elapsed time: 12.296260s
